In [ ]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
'''
@文件        :09.homoGenes.ipynb
@说明        :
@时间        :2023/02/25 19:40:24
@作者        :Wu Baosheng
@版本        :1.0
This script is used to find the homologous genes between two species. RBH for two species is ortholog
'''

In [1]:
import os,re,sys
import pandas as pd
import scanpy as sc
import numpy as np
os.chdir('H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/maps')

In [2]:
# def RBH(data1="",data2="",e_value=1e-20,score=80):
#     sp1 = os.path.basename(data1).split("_")[0]
#     sp2 = os.path.basename(data2).split("_")[0]
#     def single_blast(data,e_value=e_value,score=score):
#         name = ["query","target","coverage","satat1","end1","gap1","start2","end2","gap2","length","e_value","score"]
#         df = pd.read_csv(data,sep="\t",header=None,index_col=None,names=name)
#         df=df[(df["e_value"] <= e_value) & (df["score"]>=score)]
#         df.sort_values(["query","e_value","score"],ascending=[True,True,False],inplace=True)
#         df.drop_duplicates(subset=["query"],keep='first',inplace=True)
#         df.sort_values(["target","e_value","score"],ascending=[True,True,False],inplace=True)
#         df.drop_duplicates(subset=["target"],keep='first',inplace=True)
#         df_t = df[["query","target"]]
#         return df_t
#     df1 = single_blast(data=data1,e_value=e_value,score=score)
#     df2 = single_blast(data=data2,e_value=e_value,score=score)
#     df2.columns = ['target','query']
#     s1=pd.merge(df1, df2, how='inner', on=['query','target'])
#     s1['query']=f"{sp1}_" + s1['query']
#     s1['target']=f"{sp2}_" + s1['target'] 
#     s2 = pd.DataFrame()
#     sp_sp = f"{sp1}_{sp2}"
#     s2[sp_sp]=s1['query'].str.cat(s1['target'],sep='-')
#     return s2

In [ ]:
# df = pd.DataFrame()
# for file in os.listdir('./'):
#     file2 = os.listdir(f'./{file}')
#     print(f'{file}/{file2[0]}')
#     df1 = RBH(data1=f'{file}/{file2[0]}',data2=f'{file}/{file2[1]}')
#     df = pd.concat([df,df1],axis=1)

In [33]:
#### 为了找mouse的同源基因，2的数据重做了，平时不用2的数据

# df_counts = pd.read_csv('H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/05.subtitution_events/Orthogroups.GeneCount.tsv',sep='\t',header=0)
df_counts = pd.read_csv('H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/05.subtitution_events/Orthogroups.GeneCount2.tsv',sep='\t',header=0)
df_counts = df_counts[df_counts['Total']<=100]
df_counts.drop(['Total'],axis=1,inplace=True)
# df_genes = pd.read_csv('H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/05.subtitution_events/Orthogroups.tsv',sep='\t',header=0)
df_genes = pd.read_csv('H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/05.subtitution_events/Orthogroups2.tsv',sep='\t',header=0)

In [48]:
###### 统计每个物种前2000个高变基因
gene_expression_dir={}
for x in df_counts.columns:
    ad=[]
    if ('Orthogroup' in x) or ('mouse' in x):
        pass
    else:
        ad=sc.read(f'H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/01.h5ad_counts/{x}_counts.h5ad')
        sc.pp.normalize_total(ad, target_sum=1e6)
        sc.pp.log1p(ad)
        sc.pp.highly_variable_genes(ad, n_top_genes=2000,batch_key='sample')
        ad = ad[:, ad.var['highly_variable']] 
        genes =ad.var_names.to_list()
        gene_expression_dir[x]=genes
        # with open(f'H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/13.homoGenes/{x}_high_variable_genes.txt','w') as f:
        #     for i in genes:
        #         f.write(i+'\n')
        # f.close()

In [49]:
gene_expression_dir.keys()

dict_keys(['bichir', 'dogshark', 'frog', 'human', 'lamprey', 'lancelet', 'lungfish'])

In [73]:
# ###统计每行中0的个数, 尽量的去排出一些独立扩张的基因
# df_counts2=df_counts.copy()
# df_counts2.drop(['lamprey','lancelet'],axis=1,inplace=True)
# df_counts2['zero']=df_counts2.apply(lambda x: x[x==0].count(),axis=1)
# df_counts2=df_counts2[df_counts2['zero']<=0]
# df_counts=df_counts[df_counts['Orthogroup'].isin(df_counts2['Orthogroup'])]

In [50]:
df_counts

,Orthogroup,bichir,dogshark,frog,human,lamprey,lancelet,lungfish,mouse
52,OG0000052,5,7,21,24,7,2,5,24
56,OG0000056,28,13,24,5,4,1,9,7
86,OG0000086,4,8,12,11,0,0,8,30
88,OG0000088,10,4,24,11,2,0,11,11
99,OG0000099,15,29,3,1,0,1,6,12
...,...,...,...,...,...,...,...,...,...
14094,OG0014094,0,1,1,0,1,0,0,0
14095,OG0014095,0,1,0,0,1,0,1,0
14096,OG0014096,0,1,1,0,0,0,1,0
14098,OG0014098,0,1,1,0,0,0,0,1


In [51]:
### 对dataframe中的每一行进行t-test
from scipy import stats
def t_test(x,ref='lancelet',alternative='greater'):
    a=x[['bichir','dogshark','frog','mouse','lungfish','lamprey']]
    b=x[ref]
    return stats.ttest_1samp(a,b,alternative=alternative)[1]

In [ ]:
df_counts['p_value']=df_counts.apply(t_test,axis=1)

In [53]:
df_counts=df_counts[df_counts['p_value']<=0.05]
df_counts.drop(['p_value'],axis=1,inplace=True)

In [54]:
info_dir={}

In [60]:
####### 1. find the genes that are more than the lancelet
info_dir = {}
for colum in df_counts.columns:
    if colum == 'lancelet' or colum=='Orthogroup':
        continue
    else:
        tmp = []
        df1 = df_counts[(df_counts[colum].astype(int) > df_counts['lancelet'].astype(int))]
        print(df1.shape)
        diff = df1[colum].sum() - df1['lancelet'].sum()
        tmp.append(diff)
        for index, row in df1.iterrows():
            a = row['lancelet']
            b = df_genes.loc[index,colum].split(', ')[a: ]
            if colum != 'mouse':
                ad_var = gene_expression_dir[colum]
            c = [i for i in b if i in ad_var]
            tmp.append(c)
        info_dir[colum] = tmp

(4767, 9)
(4507, 9)
(4435, 9)
(4575, 9)
(2206, 9)
(4683, 9)
(4637, 9)


In [61]:
homo_dir = {}
for key in info_dir.keys():
    homo = info_dir[key][0]
    tmp = []
    counter = 0
    for j in info_dir[key][1: ]:
        counter+=len(j)
    homo_dir[key]=[homo,counter]
    

In [62]:
homo_dir

{'bichir': [6760, 0],
 'dogshark': [6391, 0],
 'frog': [6207, 716],
 'human': [6118, 843],
 'lamprey': [3239, 616],
 'lungfish': [6505, 767],
 'mouse': [6179, 0]}

In [63]:
df_homo = pd.DataFrame(homo_dir).T
df_homo.columns=['homo_genes','subtitution_genes']
df_homo=df_homo.reindex(['lamprey','dogshark','bichir','lungfish','frog','mouse'])

In [64]:
# for key in info_dir.keys():
#     table =  open(f'H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/13.homoGenes/{key}.homoGenes.txt','w')
#     homo = info_dir[key][0]
#     tmp = []
#     counter = 0
#     for j in info_dir[key][1: ]:
#         tmp= tmp+j
#     table.write('\n'.join(tmp))
#     table.close()
#     # homo_dir[key]=[homo,counter]
####### 1. find the genes that are more than the lancelet
for colum in df_counts.columns:
    if colum == 'lancelet' or colum=='Orthogroup':
        continue
    if colum == 'mouse':
        tmp = []
        df1 = df_counts[(df_counts[colum].astype(int) > df_counts['lancelet'].astype(int))]
        df2 = df_genes[df_genes['Orthogroup'].isin(df1['Orthogroup'])]
        df2.to_csv(f'H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/13.homoGenes/mouse.homoGenes.txt',sep='\t',header=True,index=True)

In [19]:
os.getcwd()

'H:\\002.singcell\\01.liver\\10.mutileSpecies\\05.SAMap\\maps'

In [281]:
def drawBar(df,out=None):
    """
    this function is used to draw the barplot
    df: dataframe. the data of barplot
    out: where to save the figure
    """
    import os,re,sys
    import seaborn as sns
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import plotly.graph_objs as go
    import plotly.express as px
    import plotly.io as pio
    fig = px.bar(df, x=df.index, y=df.columns,barmode='group')  ###color_discrete_sequence=px.colors.qualitative.Set1,
    fig.update_layout(title="WGD events in different species",xaxis_title="Species",yaxis_title="WGD events")
    # fig = go.Figure()
    # fig.add_bar(x=df_homo.index,y=df_homo['homo_genes'],text=df_homo['homo_genes'], textposition='auto')
    # fig.add_bar(x=df_homo.index,y=df_homo['subtitution_genes'])
    if out==None:
        pass
    else:
        pio.write_image(fig,out, format='pdf')
    return fig.show()

In [282]:
# drawBar(df=df_homo,out='H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/05.subtitution_events/WGD_events.pdf')
drawBar(df=df_homo)

In [65]:
### 对dataframe中的每一行进行t-test
from scipy import stats
def t_test2(x,ref='lamprey',alternative='greater'):
    a=x[['bichir','dogshark','frog','human','lungfish']]
    b=x[ref]
    return stats.ttest_1samp(a,b,alternative=alternative)[1]

In [66]:
df_counts2=df_counts.copy()
df_counts2['q_value']=df_counts2.apply(t_test2,axis=1)

In [67]:
df_counts2=df_counts2[df_counts2['q_value']<=0.05]

In [68]:
df_counts2.drop(['q_value'],axis=1,inplace=True)

In [ ]:
##### WGD for Gnathostomata R2
info_dir2 = {}
for colum in df_counts2.columns:
    if colum == 'lancelet' or colum=='Orthogroup' or colum=='lamprey':
        continue
    else:
        tmp = []
        df1 = df_counts[(df_counts[colum].astype(int) > df_counts['lamprey'].astype(int))]
        if colum=='mouse':
            df3 = df_genes[df_genes.index.isin(df1.index)]
            df3.to_csv('H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/13.homoGenes/mouse_R2.txt',sep='\t')
        print(df1.shape)
        diff = df1[colum].sum() - df1['lamprey'].sum()
        tmp.append(diff)
        for index, row in df1.iterrows():
            a = row['lamprey']
            b = df_genes.loc[index,colum].split(', ')[a: ]
            ad_var = gene_expression_dir[colum]
            c = [i for i in b if i in ad_var]
            tmp.append(c)
        info_dir2[colum] = tmp

In [27]:
df_genes

,Orthogroup,bichir,dogshark,frog,human,lamprey,lancelet,lungfish
0,OG0000000,"BIRCIR013649, BIRCIR017324, BIRCIR019025","PRE00085, PRE00200, PRE00776, PRE00941, PRE009...","FROG000182, FROG000405, FROG000569, FROG000779...","CENPU, SLFNL1","QSM066218, QSM135391","LOC118421383, LOC118425841","GENE5191, GENE7146"
1,OG0000001,BIRCIR002416,NaN,"FROG000019, FROG000305, FROG000363, FROG000689...",NaN,NaN,NaN,NaN
2,OG0000002,NaN,"PRE01134, PRE05126","FROG000335, FROG001274, FROG001334, FROG001335...",NaN,NaN,NaN,NaN
3,OG0000003,BIRCIR020077,NaN,"FROG001880, FROG001882, FROG001884, FROG001885...",NaN,NaN,NaN,NaN
4,OG0000004,"B0045, BIRCIR000058, BIRCIR001464, BIRCIR00184...","PRE00184, PRE17108, PRE17826, PRE17877, PRE179...","F0008, F0054, F0097, F0137, F0153, F0203, F022...","RP13-582O9_6, YY2, ZFP41, ZNF22, ZNF296, ZNF32...","QSM013696, QSM025311, QSM029081, QSM033280, QS...","LOC118403463, LOC118403768, LOC118403783, LOC1...","GENE11289, GENE11700, GENE11720, GENE11833, GE..."
...,...,...,...,...,...,...,...,...
15144,OG0015144,NaN,NaN,NaN,NaN,NaN,NaN,"GENE16969, GENE17427"
15145,OG0015145,NaN,NaN,NaN,NaN,NaN,NaN,"GENE17793, GENE18149"
15146,OG0015146,NaN,NaN,NaN,NaN,NaN,NaN,"GENE18775, GENE19855"
15147,OG0015147,NaN,NaN,NaN,NaN,NaN,NaN,"GENE18876, GENE19000"


In [23]:
homo_dir2 = {}
for key in info_dir2.keys():
    homo = info_dir2[key][0]
    tmp = []
    counter = 0
    for j in info_dir2[key][1: ]:
        counter+=len(j)
    homo_dir2[key]=[homo,counter]
df_homo2 = pd.DataFrame(homo_dir2).T
df_homo2.columns=['homo_genes','subtitution_genes']
df_homo2=df_homo2.reindex(['dogshark','bichir','lungfish','frog','human'])

In [26]:
info_dir2['human'][1: ]

[[],
 [],
 ['KRT7', 'KRT8', 'KRT86'],
 ['ASGR1', 'CLEC10A', 'CLEC4A', 'CLEC4C', 'CLEC4D', 'CLEC4E'],
 [],
 [],
 ['CDH19', 'CDH5', 'CDH6'],
 ['HBA2', 'HBB'],
 ['S1PR5'],
 ['TUBB6'],
 ['VSIG2'],
 ['VIM'],
 ['C5AR2', 'CMKLR1'],
 [],
 [],
 ['CYP2A6', 'CYP2A7', 'CYP2C9', 'CYP2E1'],
 ['SULT1C4'],
 [],
 [],
 ['HSPA6'],
 [],
 ['IL18RAP', 'IL1R1', 'IL1RL1'],
 [],
 [],
 [],
 [],
 ['GBP5'],
 [],
 ['ADH1B', 'ADH4'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['GYPC'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['KCNJ16', 'KCNJ2'],
 [],
 [],
 ['EFNA5'],
 [],
 [],
 [],
 [],
 [],
 ['MASP1'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['SLC2A3'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['FAM110D'],
 [],
 [],
 ['AQP9'],
 ['CD300E', 'TMIGD3', 'TREM1', 'TREM2'],
 ['GZMA', 'GZMK'],
 [],
 [],
 [],
 [],
 ['LGALS9'],
 ['FOSB', 'FOSL1'],
 [],
 ['SLCO1B3'],
 [],
 [],
 [],
 [],
 ['VPREB3'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['AKR1C1', 'AKR1C3', 'AKR1C4'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['DPYSL3'],
 [],
 [],
 [],
 

In [24]:
df_homo2

,homo_genes,subtitution_genes
dogshark,3437,396
bichir,3870,586
lungfish,3872,486
frog,3712,413
human,3550,497


In [283]:
# drawBar(df=df_homo2,out='H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/05.subtitution_events/WGD_events_R2.pdf')
drawBar(df=df_homo2)

In [25]:
df_homo

,homo_genes,subtitution_genes
lamprey,3231,619
dogshark,5788,698
bichir,6413,942
lungfish,6404,751
frog,6148,696
human,5980,799


In [227]:
df_homo2.loc['lamprey']=0

In [231]:
homo1=df_homo['homo_genes']-df_homo2['homo_genes']

In [230]:
sub1=df_homo['subtitution_genes']-df_homo2['subtitution_genes']

In [234]:
R1=pd.concat([homo1,sub1],axis=1)

In [235]:
R1

,homo_genes,subtitution_genes
bichir,2543,2272
dogshark,2351,2251
frog,2436,1857
human,2430,2174
lamprey,3231,2784
lungfish,2532,2484


In [238]:
df_homo2.drop(['lamprey'],axis=0,inplace=True)
df_homo2.index=['dogshark2','bichir2','lungfish2','frog2','human2']

In [239]:
df_homo2

,homo_genes,subtitution_genes
dogshark2,3437,3197
bichir2,3870,3311
lungfish2,3872,3790
frog2,3712,2637
human2,3550,2917


In [242]:
df_final = pd.concat([R1,df_homo2],axis=0)

In [244]:
df_final.index

Index(['bichir', 'dogshark', 'frog', 'human', 'lamprey', 'lungfish',
       'dogshark2', 'bichir2', 'lungfish2', 'frog2', 'human2'],
      dtype='object')

In [246]:
df_final=df_final.reindex(['lamprey','dogshark','bichir','lungfish','frog','human','dogshark2','bichir2','lungfish2','frog2','human2'])

In [ ]:
# drawBar(df=df_final,out='H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/05.subtitution_events/WGD_events_R1_R2.pdf')
drawBar(df=df_final)

In [290]:
df_final

,homo_genes,subtitution_genes,species
0,3231,2784,lamprey
1,2351,2251,dogshark
2,2543,2272,bichir
3,2532,2484,lungfish
4,2436,1857,frog
5,2430,2174,human
6,3437,3197,dogshark
7,3870,3311,bichir
8,3872,3790,lungfish
9,3712,2637,frog


In [256]:
df_final['species']=df_final.index

In [259]:
df_final.index=range(0,len(df_final))

In [264]:
df_final['species']=['lamprey', 'dogshark', 'bichir', 'lungfish', 'frog', 'human', 'dogshark', 'bichir', 'lungfish', 'frog', 'human']

In [288]:
import os,re,sys
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
fig = px.bar(df_final,y=['homo_genes','subtitution_genes'],barmode='group')
pio.write_image(fig,'H:/002.singcell/01.liver/10.mutileSpecies/05.SAMap/05.subtitution_events/WGD_events_R1_R2.pdf',format='pdf')
fig.show()

In [267]:
df_final

,homo_genes,subtitution_genes,species
0,3231,2784,lamprey
1,2351,2251,dogshark
2,2543,2272,bichir
3,2532,2484,lungfish
4,2436,1857,frog
5,2430,2174,human
6,3437,3197,dogshark
7,3870,3311,bichir
8,3872,3790,lungfish
9,3712,2637,frog


In [14]:
info_dir['human']

[5980,
 ['KRT18', 'KRT19'],
 [],
 ['KRT7', 'KRT8', 'KRT86'],
 ['CLEC10A', 'CLEC4A', 'CLEC4C', 'CLEC4D', 'CLEC4E'],
 [],
 ['EPHA4'],
 ['CDH11', 'CDH19', 'CDH5', 'CDH6'],
 ['CYGB', 'HBA1', 'HBA2', 'HBB'],
 [],
 ['SEMA3B', 'SEMA3C', 'SEMA3E'],
 ['S1PR1', 'S1PR3', 'S1PR5'],
 ['TUBB4A', 'TUBB6'],
 ['VSIG2'],
 ['HIST1H1D'],
 ['VIM'],
 [],
 ['C5AR1', 'C5AR2', 'CMKLR1'],
 [],
 [],
 [],
 ['CYP2A6', 'CYP2A7', 'CYP2C9', 'CYP2E1'],
 ['SULT1C4'],
 [],
 [],
 [],
 ['HSPA6'],
 [],
 [],
 ['IL18RAP', 'IL1R1', 'IL1RL1'],
 [],
 [],
 [],
 [],
 ['GBP4', 'GBP5'],
 [],
 [],
 ['ADAM8'],
 ['CPB2'],
 ['ADH1B', 'ADH4'],
 ['ATP1A1'],
 [],
 [],
 [],
 [],
 [],
 ['COL1A2', 'COL3A1'],
 [],
 ['CSF3R', 'IL6ST'],
 [],
 ['CADM3', 'GYPC'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['KCNJ16', 'KCNJ2'],
 [],
 ['ATP1B2', 'ATP1B3'],
 ['EFNA5'],
 ['GJA4', 'GJA5'],
 [],
 [],
 [],
 ['SLC16A3'],
 [],
 [],
 [],
 [],
 ['NOV'],
 [],
 ['C1S', 'MASP1'],
 [],
 [],
 [],
 [],
 [],
 ['ARHGEF28'],
 [],
 [],
 ['ITGA5'],
 [

In [13]:
gene_expression_dir

{'bichir': ['B0021',
  'B0029',
  'B0116',
  'B0136',
  'B0176',
  'B0263',
  'B0287',
  'B0306',
  'BIRCIR000015',
  'BIRCIR000017',
  'BIRCIR000024',
  'BIRCIR000031',
  'BIRCIR000034',
  'BIRCIR000059',
  'BIRCIR000061',
  'BIRCIR000084',
  'BIRCIR000090',
  'BIRCIR000098',
  'BIRCIR000103',
  'BIRCIR000106',
  'BIRCIR000111',
  'BIRCIR000115',
  'BIRCIR000131',
  'BIRCIR000143',
  'BIRCIR000174',
  'BIRCIR000186',
  'BIRCIR000188',
  'BIRCIR000240',
  'BIRCIR000259',
  'BIRCIR000301',
  'BIRCIR000320',
  'BIRCIR000323',
  'BIRCIR000324',
  'BIRCIR000335',
  'BIRCIR000344',
  'BIRCIR000346',
  'BIRCIR000347',
  'BIRCIR000353',
  'BIRCIR000354',
  'BIRCIR000356',
  'BIRCIR000362',
  'BIRCIR000368',
  'BIRCIR000383',
  'BIRCIR000398',
  'BIRCIR000401',
  'BIRCIR000422',
  'BIRCIR000443',
  'BIRCIR000444',
  'BIRCIR000445',
  'BIRCIR000457',
  'BIRCIR000472',
  'BIRCIR000473',
  'BIRCIR000478',
  'BIRCIR000479',
  'BIRCIR000495',
  'BIRCIR000513',
  'BIRCIR000537',
  'BIRCIR000553',
  